## Script responsável por descompactar o ZIP

- Antes de executar esse script, fazer download da base do DOU no link: https://www.in.gov.br/acesso-a-informacao/dados-abertos/base-de-dados
- Há um arquivo zip para cada ano/mês/seção
- Salvar no subdiretório \downloads\Secao0<Número da secao>\ano<com 4 dígitos>. Exemplo: \downloads\Secao02\2024
- Processamento deste script:
  - Descompacta todos os arquivos zips existentes no subdiretório downloads\Secao0<Número da secao>
  - Faz a leitura do XML
  - Cria a coluna Texto a partir da TextHTML, removento das tags html do corpo da portaria
  - Cria a coluna Ano com o ano da publicação
  - Cria a coluna Mês com o mês da publicação
  - Cria a coluna Total_palavras, que exibe o total de palavras da coluna Texto
  - Salva a saída no arquivo **.\saida\DouSecao0<No Seção>_completo.parquet**
---


## Bibliotecas

In [1]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import io

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

In [3]:
def process_zip(zip_path):

    print(zip_path)
    # Lista para armazenar os dados dos artigos
    articles_data = []

    # Abre o arquivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Itera sobre cada arquivo no ZIP
        for file_name in zip_ref.namelist():
            if file_name.endswith('.xml'):
                try:
                    # Abre o arquivo XML
                    with zip_ref.open(file_name) as file:
                        # Lê o conteúdo do arquivo
                        content = file.read()
                        # Verifica se o conteúdo não está vazio
                        if content.strip():
                            # Parse do conteúdo XML
                            tree = ET.ElementTree(ET.fromstring(content))
                            root = tree.getroot()

                            # Itera sobre todos os elementos <article>
                            for article in root.findall('article'):
                                if article is not None:
                                    # Extrai os atributos do <article>
                                    article_data = article.attrib

                                    # Extrai os conteúdos dos elementos dentro de <body>
                                    body = article.find('body')
                                    if body is not None:
                                        for child in body:
                                            # Usa o nome do elemento como chave e seu texto como valor
                                            article_data[child.tag] = child.text.strip() if child.text else None

                                    # Adiciona o nome do arquivo aos dados do artigo
                                    article_data['file_name'] = file_name
                                    # Adiciona o nome do arquivo ZIP aos dados do artigo
                                    article_data['zip_name'] = os.path.basename(zip_path)
                                    articles_data.append(article_data)
                except ET.ParseError:
                    print(f"Erro ao analisar XML no arquivo {file_name} no ZIP {zip_path}")
                except Exception as e:
                    print(f"Erro ao processar o arquivo {file_name} no ZIP {zip_path}: {e}")

    return articles_data

In [4]:
def process_all_zips_in_directory(directory):

    print(directory)
    # Lista para armazenar os dados de todos os artigos
    all_articles_data = []

    # Itera sobre todos os arquivos no diretório e subdiretórios
    for root, dirs, files in os.walk(directory):
        print(f'Processando diretório {root}...')
        for file_name in files:
            if file_name.endswith('.zip'):
                zip_path = os.path.join(root, file_name)
                articles_data = process_zip(zip_path)
                all_articles_data.extend(articles_data)

    # Cria um DataFrame a partir dos dados de todos os artigos
    df = pd.DataFrame(all_articles_data)
    return df

In [5]:
douItem = input("Informe a seção: 1, 2 ou 3")
secao = f'Secao0{douItem}'

# Caminho para o diretório contendo os arquivos ZIP
directory_path = os.path.join('./downloads/', secao)

# Processa todos os ZIPs no diretório e obtém o DataFrame
df = process_all_zips_in_directory(directory_path)

# Renomeia a coluna Texto
df.rename(columns={'Texto': 'TextoHTML'}, inplace=True)

# Exibe o DataFrame
df.shape

Informe a seção: 1, 2 ou 3 2


./downloads/Secao02
Processando diretório ./downloads/Secao02...
Processando diretório ./downloads/Secao02\2022...
./downloads/Secao02\2022\S02012022.zip
./downloads/Secao02\2022\S02022022.zip
./downloads/Secao02\2022\S02032022.zip
./downloads/Secao02\2022\S02042022.zip
./downloads/Secao02\2022\S02052022.zip
./downloads/Secao02\2022\S02062022.zip
./downloads/Secao02\2022\S02072022.zip
./downloads/Secao02\2022\S02082022.zip
./downloads/Secao02\2022\S02092022.zip
./downloads/Secao02\2022\S02102022.zip
./downloads/Secao02\2022\S02112022.zip
./downloads/Secao02\2022\S02122022.zip
Processando diretório ./downloads/Secao02\2023...
./downloads/Secao02\2023\S02012023.zip
./downloads/Secao02\2023\S02022023.zip
./downloads/Secao02\2023\S02032023.zip
./downloads/Secao02\2023\S02042023.zip
./downloads/Secao02\2023\S02052023.zip
./downloads/Secao02\2023\S02062023.zip
./downloads/Secao02\2023\S02072023.zip
./downloads/Secao02\2023\S02082023.zip
./downloads/Secao02\2023\S02092023.zip
./downloads/Seca

(497325, 27)

In [6]:
# Salva o arquivo completo
df.to_parquet(f'./saida/01_DOU{secao}_completo.parquet', engine='pyarrow', index=False)

In [7]:
df.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,highlightimagename,idMateria,Identifica,Data,Ementa,Titulo,SubTitulo,TextoHTML,file_name,zip_name
0,23628592,trt9a174-2021 PRT DG 201,6760729,DO2,Portaria,03/01/2022,00046:00048:00000:00000:00000:00000:00000:0000...,Poder Judiciário/Tribunal Regional do Trabalho...,12,,...,,14007666,"PORTARIA DG Nº 201, DE 16 DE NOVEMBRO DE 2021",None,None,None,None,"<p class=""identifica"">PORTARIA DG Nº 201, DE 1...",S02012022/529_20220103_14007666.xml,S02012022.zip
1,23628441,PT CEAB N 1006 ANA CECILIA LEUTW,6790787,DO2,Portaria,03/01/2022,00027:00017:00275:00001:00000:00000:00000:0000...,Ministério do Trabalho e Previdência/Instituto...,12,,...,,14050542,"PORTARIA CEAB - RPPS/INSS Nº 1.006, DE 30 DE N...",None,None,None,None,"<p class=""identifica"">PORTARIA CEAB - RPPS/INS...",S02012022/529_20220103_14050542.xml,S02012022.zip
2,23628934,PT1019CEAB_03-01,6803814,DO2,Portaria,03/01/2022,00027:00017:00275:00001:00000:00000:00000:0000...,Ministério do Trabalho e Previdência/Instituto...,12,,...,,14068872,"PORTARIA CEAB - RPPS/INSS Nº 1.019, DE 2 DE DE...",None,None,None,None,"<p class=""identifica"">PORTARIA CEAB - RPPS/INS...",S02012022/529_20220103_14068872.xml,S02012022.zip
3,23628444,PORTARIA NA 353,6822161,DO2,Portaria,03/01/2022,00015:00063:00000:00000:00000:00000:00000:0000...,Ministério da Economia/Fundação Escola Naciona...,24,,...,,14095736,"PORTARIA Nº 353, DE 13 DE DEZEMBRO DE 2021",None,None,None,None,"<p class=""identifica"">PORTARIA Nº 353, DE 13 D...",S02012022/529_20220103_14095736.xml,S02012022.zip
4,23628486,PORTARIA 2050,6822393,DO2,Portaria,03/01/2022,00018:00092:00000:00000:00000:00000:00000:0000...,Ministério da Educação/Universidade Federal Fl...,12,,...,,14096096,"PORTARIA Nº 2.050, de 13 de dezembro de 2021",None,None,None,None,"<p></p><p class=""identifica"">PORTARIA Nº 2.050...",S02012022/529_20220103_14096096.xml,S02012022.zip
